#### Llama 2

In [53]:
!pip install transformers datasets
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write)

In [54]:
from transformers import AutoTokenizer
import transformers
import torch
import json

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [57]:
with open('all_scores.json', 'r') as file:
    data = json.load(file)

In [58]:
def extra_emotion(valence, arousal):
  emotion = f"""
  Take into account my current emotional state without explicitly mentioning it. Valence is ranging from negative to positive. Arousal is intensity of emotion. Both ranging from 1-9.
  Arousal: {arousal}
  Valence: {valence}
  """
  return emotion

In [60]:
def make_texts(stage_num, data):
    stage = data[f'stage_{stage_num}']
    print(f"Generating texts for stage{stage_num}")

    prompt_baseline = stage['prompt']
    prompt_postive = stage['prompt'] + " " + extra_emotion(stage['valence_pos'], stage['arousal_pos'])
    prompt_negative = stage['prompt'] + " " + extra_emotion(stage['valence_neg'], stage['arousal_neg'])

    print('Generating baseline')
    text_baseline = do_llama_seq(prompt_baseline)
    print('Generating positive')
    text_postive = do_llama_seq(prompt_postive)
    print('Generating negative')
    text_negative = do_llama_seq(prompt_negative)

    data[f'stage_{stage_num}']['text_bas'] = text_baseline
    data[f'stage_{stage_num}']['text_pos'] = text_postive
    data[f'stage_{stage_num}']['text_neg'] = text_negative



In [62]:
def do_llama_seq(prompt):
  sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=10,
    max_length= 200,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id

    )
  return sequences[0]['generated_text']

In [63]:
for i in range(1,6):
  make_texts(i, data)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generating texts for stage1
Generating baseline


KeyboardInterrupt: 

In [23]:
texts = json.dumps(data, indent=2)
with open('text_and_scores.json', 'w') as f:
  f.write(texts)

In [25]:
print(texts['stage_3']['text_pos'])